In [1]:
from github import Github
import plotly.graph_objects as go
import pandas as pd 
import numpy as np 

In [2]:
# Set up github access and get members from an organization
pd.set_option('display.max_rows', None)
g = Github("6dec4a753453cdbf66d526d8a646004deea5d11c")
df = pd.DataFrame(columns=['Member'])
memberList = []
for member in g.get_organization("PyGithub").get_members():
    memberList.append(member.login)

In [24]:
# Get all of the member's repositories
df = pd.DataFrame(columns=['Username','Repository','DateCreated','Language', 'Size'])
for member in memberList:
    currentUser = g.get_user(member)
    for repo in currentUser.get_repos():
        new_row = {'Username': member.strip(), 'Repository': repo.name.strip(), 'DateCreated': repo.created_at.year, 'Language':repo.language, 'Size':repo.size, 'Contributors':repo.get_contributors().totalCount}
        df = df.append(new_row, ignore_index=True)

,Username,Repository,DateCreated,Language,Size,Contributors
0,adamtheturtle,ACDTools,2017,Shell,77,2.0
1,adamtheturtle,acd_cli,2017,Python,737,23.0
2,adamtheturtle,addict,2017,Python,99,16.0
3,adamtheturtle,Alfred-Extensions,2017,None,113,1.0
4,adamtheturtle,attrs,2016,Python,368,20.0
5,adamtheturtle,boggle-solver,2015,Python,864,1.0
6,adamtheturtle,brew,2019,Ruby,30046,407.0
7,adamtheturtle,characteristic,2014,Python,254,5.0
8,adamtheturtle,chroot_tasker,2016,Python,2370,2.0
9,adamtheturtle,click,2017,Python,1458,92.0


In [28]:
df = df.sort_values(by=['Username', 'DateCreated'])
df = df[(df[['Size','Contributors']] != 0).all(axis=1)]
df = df[df.Language != 'None']
for member in memberList:
    mask = (df['Username'] == member)
    df['DateCreated'][mask] = df['DateCreated'] - (df.loc[df['Username'] == member])['DateCreated'].min()

/Users/jackengels1/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Username,Repository,DateCreated,Language,Size,Contributors
197,JPWKU,KandR,0,C,104,1.0
229,JPWKU,tdd_tut,0,Python,136,1.0
110,JPWKU,biblion,1,Python,241,6.0
112,JPWKU,BTree,1,Java,112,1.0
175,JPWKU,emacs,1,Emacs Lisp,808,1.0
196,JPWKU,kaleo,1,Python,250,5.0
198,JPWKU,kitchen-ec2-test,1,Shell,108,1.0
199,JPWKU,learn_erlang,1,Erlang,124,1.0
200,JPWKU,mezzanine,1,Python,40295,186.0
201,JPWKU,microblog,1,Python,184,1.0


In [78]:
tempArray = df.loc[df['DateCreated'] == 2]
tempArray = tempArray.drop(['Username', 'Repository',"DateCreated"], axis=1)
yearArray = pd.DataFrame(columns=['Language','Repositories','Average Size','Contributors'])
yearArray['Language'] = tempArray.Language.unique()
uniqueValues = tempArray.Language.unique()
for language in uniqueValues:
    mask = (yearArray['Language'] == language)
    yearArray['Repositories'][mask] = tempArray.loc[tempArray['Language'] == language].Language.count()
    yearArray['Average Size'][mask] = tempArray.loc[tempArray['Language'] == language].Size.mean()
    yearArray['Contributors'][mask] = tempArray.loc[tempArray['Language'] == language].Contributors.mean()
yearArray

,Language,Repositories,Average Size,Contributors
0,Python,17,3503.53,15.4706
1,Shell,1,479,1
2,C,2,17994.5,62.5
3,Go,1,116485,428
4,JavaScript,3,492.667,1
5,Objective-C,1,2693,23
6,None,NaN,NaN,NaN
